# Домашняя работа

1. Теперь решаем задачу регрессии - предскажем цены на недвижимость. Использовать датасет https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data (train.csv)
2. Построить случайный лес, вывести важность признаков
3. Обучить стекинг как минимум 3х моделей, использовать хотя бы 1 линейную модель и 1 нелинейную
4. В качестве решения: Jupyter notebook с кодом, комментариями и графиками

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

In [2]:
#Формируем датафрейм
df_test = pd.read_csv('data/test.csv') #тестовые данные
df_sample_submission = pd.read_csv('data/sample_submission.csv') #цены по df_test
df_trial = pd.read_csv('data/train.csv') #тренировочные даные
#Основной датафрейм
df = pd.concat([df_trial, df_test.merge( df_sample_submission )], axis = 0, ignore_index = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             2919 non-null   int64  
 1   MSSubClass     2919 non-null   int64  
 2   MSZoning       2915 non-null   object 
 3   LotFrontage    2433 non-null   float64
 4   LotArea        2919 non-null   int64  
 5   Street         2919 non-null   object 
 6   Alley          198 non-null    object 
 7   LotShape       2919 non-null   object 
 8   LandContour    2919 non-null   object 
 9   Utilities      2917 non-null   object 
 10  LotConfig      2919 non-null   object 
 11  LandSlope      2919 non-null   object 
 12  Neighborhood   2919 non-null   object 
 13  Condition1     2919 non-null   object 
 14  Condition2     2919 non-null   object 
 15  BldgType       2919 non-null   object 
 16  HouseStyle     2919 non-null   object 
 17  OverallQual    2919 non-null   int64  
 18  OverallC

In [3]:
#Класс для очистики датафрейм 
#Категориальные данные преобразуем по O H E (унимодальное кодирование)
#Пропуски заменяем округленным до целого средним

class cls2task_prepdre_df:
    
    ret_df = pd.DataFrame()
    
    def __init__(self, df_):
        self.df      = df_
        #list(df.dtypes[df.dtypes == object].index)
        self.df_num = df_.select_dtypes(exclude=['object']).copy()
        self.df_obj = df_.select_dtypes(include=['object']).copy()
    
    def ohe(self):
        ret = pd.DataFrame()
        for col in self.df_obj.columns:
            _ = pd.get_dummies(self.df_obj[col], prefix=col, drop_first=False)
            ret =  pd.concat([ret, _], axis=1)
        return ret
    
    def set_mean(self):
        for col in self.ret_df.columns:
            if self.ret_df[col].isna().sum() > 0:
                _ = self.ret_df[[col]].mean().round(0)
                self.ret_df.loc[ self.ret_df[ self.ret_df[col].isna()].index, col] = _[0]                
        return 0
    
    def main(self):
        self.ret_df = pd.concat([self.df_num, self.ohe()], axis=1)
        self.set_mean()
        return self.ret_df
    

# Случайный лес


**Важные гиперпараметры алгоритма**

a. Параметры деревьев
1. criterion - критерий построения дерева
2. max_depth - максимальная глубина дерева (обычно 10-20, больше глубина -> больше риск переобучения)
3. min_samples_leaf - минимальное число объектов в листе (обычно 20+, больше объектов -> меньше риск переобучения)

b. Параметры леса
1. n_estimators - кол-во деревьев (чем больше тем лучше)
2. max_features - число признаков случайного подпространства
3. n_jobs - кол-во потоков для одновременного построения деревьев (большая прибавка к скорости на многоядерных процах)

In [4]:
#Обрабатываем основной датафрейм
transform_df = cls2task_prepdre_df(df.iloc[:,1:-1]) #из обработки иключаем 1 (Id) и послденее поле (SalePrice)
transform_df.main()

#Признаковое пространство
X = transform_df.ret_df
#Target-значение (цена)
y = df.iloc[:,-1].round(0)
#Делим выборку на тренировочную и тестовую (в отбор включаем все значимые призанки по условию значимости  > 0.00001%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
#Создаем и обучаем модель
model_rf = RandomForestRegressor(n_estimators=100, max_depth=25, min_samples_leaf=25, max_features=0.5, n_jobs=-1)
model_rf.fit(X_train, y_train)
print(f'Качество (R2): {model_rf.score(X_test , y_test)}\n' )

Качество (R2): 0.4296632767253832



In [5]:
#Важность признака (%)
ia_df = pd.DataFrame( {'Attribute':X.columns, \
                       'Importances':100*model_rf.feature_importances_} ).\
           sort_values(by = 'Importances', ascending = False).reset_index()

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also  
    print( ia_df[ia_df['Importances'] > 0.00001] )


    index             Attribute  Importances
0      15             GrLivArea    30.786561
1       3           OverallQual    23.106732
2      25            GarageCars     7.700062
3      11           TotalBsmtSF     7.482305
4      26            GarageArea     4.291740
5       2               LotArea     4.202292
6      12              1stFlrSF     3.109908
7      22          TotRmsAbvGrd     2.868107
8      18              FullBath     1.794582
9       8            BsmtFinSF1     1.787734
10     13              2ndFlrSF     1.675687
11    224        KitchenQual_Ex     1.092642
12      5             YearBuilt     1.017512
13     20          BedroomAbvGr     0.781985
14     34                MoSold     0.688081
15      1           LotFrontage     0.620826
16      6          YearRemodAdd     0.604947
17      7            MasVnrArea     0.529936
18    182           BsmtQual_Ex     0.496581
19     10             BsmtUnfSF     0.487388
20     27            WoodDeckSF     0.452754
21      0 

# Стекинг 3-х моделей

In [6]:
# Библиотеки машинного обучения
from sklearn.ensemble import StackingRegressor #Стекинг
from sklearn.linear_model import RidgeCV #Линейная модель
from sklearn.neighbors import KNeighborsRegressor #К-ближайших соседей
from sklearn.tree import DecisionTreeRegressor #Дерево решений

In [7]:
#Ансамбль моделей
regressor = StackingRegressor(
    [
        ('rdg', RidgeCV()),
        ('dt',   DecisionTreeRegressor()),
        ('kn',   KNeighborsRegressor()),
    ],
RidgeCV())
#Делим выборку на тренировочную и тестовую (в отбор включаем все значимые призанки по условию значимости  > 0.00001%)
X_train, X_test, y_train, y_test = train_test_split(X[ia_df[ia_df['Importances'] > 1 ]['Attribute']], y, \
                                                    test_size=0.3, random_state = 42)

#Нормирование данных
scaler = StandardScaler()
scaler.fit(X_train)
X_train_std = scaler.transform(X_train)
#Обучение ансабля
regressor.fit(X_train_std, y_train)

StackingRegressor(estimators=[('rdg',
                               RidgeCV(alphas=array([ 0.1,  1. , 10. ]))),
                              ('dt', DecisionTreeRegressor()),
                              ('kn', KNeighborsRegressor())],
                  final_estimator=RidgeCV(alphas=array([ 0.1,  1. , 10. ])))

In [8]:
X_test_std = scaler.transform(X_test)
y_pred = regressor.predict(X_test_std)
print(f'Качество (R2): { regressor.score(X_test_std, y_test)}\n' )
print('Прогноз цен на недвижимость (сравнение c фактом):')
list( zip(y_test, y_pred.round(0) ) )

Качество (R2): 0.5093757712976595

Прогноз цен на недвижимость (сравнение c фактом):


[(180097.0, 150568.0),
 (192687.0, 192884.0),
 (158152.0, 143628.0),
 (187500.0, 199232.0),
 (172024.0, 199560.0),
 (311872.0, 229430.0),
 (154035.0, 117899.0),
 (260000.0, 195639.0),
 (185998.0, 142150.0),
 (181613.0, 177907.0),
 (176515.0, 171560.0),
 (201691.0, 208053.0),
 (176488.0, 162554.0),
 (177548.0, 154848.0),
 (149900.0, 150853.0),
 (207500.0, 198057.0),
 (128500.0, 159109.0),
 (189000.0, 189231.0),
 (128000.0, 158301.0),
 (180117.0, 176320.0),
 (268000.0, 223249.0),
 (34900.0, 114741.0),
 (168176.0, 149264.0),
 (271000.0, 208369.0),
 (224324.0, 201321.0),
 (180258.0, 182003.0),
 (301000.0, 233740.0),
 (159271.0, 133107.0),
 (228156.0, 248789.0),
 (151400.0, 210282.0),
 (185734.0, 214296.0),
 (165625.0, 143832.0),
 (194543.0, 202914.0),
 (180000.0, 175750.0),
 (186599.0, 208000.0),
 (176270.0, 179535.0),
 (143000.0, 172240.0),
 (181872.0, 142584.0),
 (166275.0, 146151.0),
 (107500.0, 137357.0),
 (188001.0, 154191.0),
 (199792.0, 210366.0),
 (260000.0, 205454.0),
 (223056.0, 